In [1]:
!pip install torch-scatter -f https://data.pyg.org/whl/torch-2.2.0+cu121.html
!pip install torch-sparse  -f https://data.pyg.org/whl/torch-2.2.0+cu121.html
!pip install torch-geometric

!pip install --use-deprecated=legacy-resolver karateclub networkx numpy pandas matplotlib scikit-learn

!pip install torch torchvision torchaudio
!pip install torch-geometric \
    -f https://data.pyg.org/whl/torch-$(python -c "import torch; print(torch.__version__)").html


!pip install optuna
!pip install karateclub

Looking in links: https://data.pyg.org/whl/torch-2.2.0+cu121.html
Looking in links: https://data.pyg.org/whl/torch-2.2.0+cu121.html
Looking in links: https://data.pyg.org/whl/torch-2.9.0+cu126.html
  Using cached numpy-1.22.4.zip (11.5 MB)
  Installing build dependencies ... done
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Getting requirements to build wheel ... error
error: subprocess-exited-with-error

× Getting requirements to build wheel did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.


In [2]:
# get statistics

import numpy as np
import torch
from torch_geometric.datasets import TUDataset
from torch_geometric.utils import is_undirected

def dataset_stats(name, root="/content/data"):
    ds = TUDataset(root=root, name=name)

    num_graphs = len(ds)
    num_classes = ds.num_classes

    node_counts = []
    edge_counts_undirected = []
    edge_counts_raw = []
    undirected_flags = []

    for g in ds:
        n = int(g.num_nodes) if g.num_nodes is not None else 0
        e = int(g.num_edges) if g.edge_index is not None else 0  # raw (directed) count in PyG
        node_counts.append(n)
        edge_counts_raw.append(e)

        # Check undirected and convert to "unique undirected edges"
        if g.edge_index is not None and g.edge_index.numel() > 0:
            und = bool(is_undirected(g.edge_index))
        else:
            und = True
        undirected_flags.append(und)

        # If undirected, PyG usually stores both directions => divide by 2
        e_und = e // 2 if und else e
        edge_counts_undirected.append(e_und)

    stats = {
        "Dataset": name,
        "#Graphs": num_graphs,
        "#Classes": num_classes,
        "Avg #Nodes": float(np.mean(node_counts)),
        "Avg #Edges (undirected)": float(np.mean(edge_counts_undirected)),
        "Pct undirected graphs": 100.0 * float(np.mean(undirected_flags)),
    }
    return stats

DATASETS = ["MUTAG", "ENZYMES", "IMDB-MULTI"]
all_stats = [dataset_stats(d) for d in DATASETS]
all_stats

/usr/local/lib/python3.12/dist-packages/torch_geometric/__init__.py:4: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: Could not load this library: /usr/local/lib/python3.12/dist-packages/torch_scatter/_version_cuda.so
  import torch_geometric.typing
/usr/local/lib/python3.12/dist-packages/torch_geometric/__init__.py:4: UserWarning: An issue occurred while importing 'torch-sparse'. Disabling its usage. Stacktrace: Could not load this library: /usr/local/lib/python3.12/dist-packages/torch_sparse/_version_cuda.so
  import torch_geometric.typing


[{'Dataset': 'MUTAG',
  '#Graphs': 188,
  '#Classes': 2,
  'Avg #Nodes': 17.930851063829788,
  'Avg #Edges (undirected)': 19.79255319148936,
  'Pct undirected graphs': 100.0},
 {'Dataset': 'ENZYMES',
  '#Graphs': 600,
  '#Classes': 6,
  'Avg #Nodes': 32.63333333333333,
  'Avg #Edges (undirected)': 62.13666666666666,
  'Pct undirected graphs': 100.0},
 {'Dataset': 'IMDB-MULTI',
  '#Graphs': 1500,
  '#Classes': 3,
  'Avg #Nodes': 13.001333333333333,
  'Avg #Edges (undirected)': 65.93533333333333,
  'Pct undirected graphs': 100.0}]

In [3]:
# Imports
import joblib
import torch
import torch.nn.functional as F
from torch.nn import Linear, Sequential, ReLU, BatchNorm1d
from torch_geometric.nn import GINConv, global_add_pool
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader
from sklearn.metrics import accuracy_score, f1_score
from torch_geometric.transforms import OneHotDegree
import optuna
import pandas as pd
import time, os, psutil
from sklearn.metrics import roc_auc_score
import numpy as np
from karateclub import NetLSD, Graph2Vec
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, cross_val_score
from torch_geometric.utils import to_networkx
import networkx as nx
import warnings
warnings.filterwarnings("ignore")


In [4]:
# Connect to google drive
from google.colab import drive
drive.mount('/content/drive')
BASE_DIR = "/content/drive/MyDrive/InformationSystems/Classification"
RESULTS_DIR = f"{BASE_DIR}/results"
MODELS_DIR = f"{BASE_DIR}/models"
EMBEDDINGS_DIR = f"{BASE_DIR}/embeddings"
CLASSIF_RESULTS_DIR = f"{BASE_DIR}/results/classification"

os.makedirs(EMBEDDINGS_DIR, exist_ok=True)
os.makedirs(RESULTS_DIR, exist_ok=True)
os.makedirs(MODELS_DIR, exist_ok=True)
os.makedirs(CLASSIF_RESULTS_DIR, exist_ok=True)

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
def make_deterministic_perm(n: int, seed: int) -> np.ndarray:
    """Deterministic permutation for dataset ordering (Fix A).

    We use this permutation everywhere we need a stable, reproducible graph order
    so that saved embeddings/labels can be aligned later (e.g., in stability.py).
    """
    return np.random.RandomState(seed).permutation(n)


In [6]:
def sanitize_embeddings(embeddings: np.ndarray) -> np.ndarray:
    """
    Replace NaN/Inf values in embeddings and ensure a clean float32 array.
    This is useful for karateclub embeddings that may occasionally produce
    unstable values on some graphs.
    """
    emb = np.asarray(embeddings, dtype=np.float32)
    # Replace NaN and +/- Inf with 0.0
    emb = np.nan_to_num(emb, nan=0.0, posinf=0.0, neginf=0.0)
    return emb

In [7]:
def filter_enzymes_graphs(graphs, labels, min_nodes: int = 3):
    """
    Special handling for ENZYMES: remove very small graphs that can cause
    numerical issues for NetLSD / Graph2Vec.
    Returns filtered (graphs, labels) and prints how many were removed.
    """
    if len(graphs) == 0:
        return graphs, labels

    mask = [g.number_of_nodes() >= min_nodes for g in graphs]
    if not any(mask):
        print("WARNING: All ENZYMES graphs would be filtered out. Skipping filtering.")
        return graphs, labels

    filtered_graphs = [g for g, keep in zip(graphs, mask) if keep]
    if isinstance(labels, np.ndarray):
       filtered_labels = labels[np.array(mask)]
    else:
        filtered_labels = [y for y, keep in zip(labels, mask) if keep]

    removed = len(graphs) - len(filtered_graphs)
    print(f"ENZYMES filtering: removed {removed} graphs with < {min_nodes} nodes, kept {len(filtered_graphs)} graphs.")
    return filtered_graphs, filtered_labels

In [8]:
# GIN Model Definition

class GIN(torch.nn.Module):
    def __init__(self, num_features, hidden_dim, num_classes, num_layers=5, dropout=0.5):
        super(GIN, self).__init__()
        layers = []
        in_dim = num_features
        for _ in range(num_layers):
            nn = Sequential(Linear(in_dim, hidden_dim), ReLU(), Linear(hidden_dim, hidden_dim))
            layers.append(GINConv(nn))
            in_dim = hidden_dim
        self.convs = torch.nn.ModuleList(layers)
        self.bns = torch.nn.ModuleList([BatchNorm1d(hidden_dim) for _ in range(num_layers)])
        self.fc1 = Linear(hidden_dim, hidden_dim)
        self.fc2 = Linear(hidden_dim, num_classes)
        self.dropout = dropout

    def forward(self, x, edge_index, batch):
        for conv, bn in zip(self.convs, self.bns):
            x = F.relu(conv(x, edge_index))
            x = bn(x)
        x = global_add_pool(x, batch)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.fc2(x)
        return x

In [9]:
# Training

def train(model, loader, optimizer, criterion):
    model.train()
    total_loss = 0
    for data in loader:
        data = data.to(DEVICE)
        optimizer.zero_grad()
        out = model(data.x, data.edge_index, data.batch)
        loss = criterion(out, data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

# Evaluation

def evaluate(model, loader, criterion):
    model.eval()
    preds, labels, probs = [], [], []
    total_loss = 0.0
    num_batches = 0

    with torch.no_grad():
        for data in loader:
            data = data.to(DEVICE)
            out = model(data.x, data.edge_index, data.batch)

            loss = criterion(out, data.y)
            total_loss += loss.item()
            num_batches += 1

            pred = out.argmax(dim=1)
            preds.extend(pred.cpu().numpy())
            labels.extend(data.y.cpu().numpy())
            probs.extend(F.softmax(out, dim=1).cpu().numpy())  # probabilities for AUC

    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='weighted')
    avg_loss = total_loss / max(1, num_batches)

    try:
        if len(np.unique(labels)) == 2:
            auc = roc_auc_score(labels, np.array(probs)[:, 1])
        else:
            auc = roc_auc_score(labels, probs, multi_class='ovr')
    except ValueError:
        auc = np.nan  # if there are not enough samples for AUC

    return acc, f1, auc, avg_loss

In [10]:
def get_gin_embeddings(model, loader):
    """Return graph-level embeddings (after global_add_pool) and labels."""
    model.eval()
    all_emb = []
    all_labels = []
    with torch.no_grad():
        for data in loader:
            data = data.to(DEVICE)
            x, edge_index, batch = data.x, data.edge_index, data.batch
            # forward μέχρι το pooling
            for conv, bn in zip(model.convs, model.bns):
                x = F.relu(conv(x, edge_index))
                x = bn(x)
            x = global_add_pool(x, batch)
            all_emb.append(x.cpu().numpy())
            all_labels.extend(data.y.cpu().numpy())
    embeddings = np.concatenate(all_emb, axis=0)
    labels = np.array(all_labels)
    return embeddings, labels

In [11]:
def run_gin_pipeline(
    dataset_name,
    seed,
    use_optuna,
    w_acc,
    w_f1,
    w_auc,
    hidden_dim,
    epochs,
    batch_size=32,
    n_trials=15,
):

    # Experiment ID (used in logs and embeddings path)
    experiment_id = str(seed)
    torch.manual_seed(seed)
    np.random.seed(seed)

    # Load dataset (Fix A: deterministic order, no implicit shuffle)
    dataset_raw = TUDataset(root='data/TUDataset', name=dataset_name)
    perm = make_deterministic_perm(len(dataset_raw), seed)
    dataset = dataset_raw[perm.tolist()]

    # If no node features use one-hot degree features
    if dataset.num_features == 0 or dataset[0].x is None:
        print("Dataset has no node features. Applying OneHotDegree transform...")

        # Find maximum degree across all graphs
        max_degree = 0
        for data in dataset:
            deg = torch.bincount(data.edge_index[0], minlength=data.num_nodes)
            max_degree = max(max_degree, int(deg.max()))

        # Apply transform
        oh_transform = OneHotDegree(max_degree=max_degree)
        dataset_raw = TUDataset(
            root='data/TUDataset',
            name=dataset_name,
            transform=oh_transform
        )

        # Apply the SAME deterministic permutation so order matches the non-feature case
        perm = make_deterministic_perm(len(dataset_raw), seed)
        dataset = dataset_raw[perm.tolist()]

        num_node_features = max_degree + 1
    else:
        num_node_features = dataset.num_features

    # Train/test split
    train_dataset = dataset[:int(0.8 * len(dataset))]
    test_dataset = dataset[int(0.8 * len(dataset)):]
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)

    print(f"Loaded dataset {dataset_name}: {len(dataset)} graphs, {num_node_features} node features, {dataset.num_classes} classes")

    def objective(trial):
        num_layers = trial.suggest_int("num_layers", 3, 6)
        dropout = trial.suggest_float("dropout", 0.0, 0.6)
        lr = trial.suggest_loguniform("lr", 1e-4, 1e-2)
        weight_decay = trial.suggest_loguniform("weight_decay", 1e-6, 1e-3)

        model = GIN(num_node_features, hidden_dim, dataset.num_classes, num_layers, dropout).to(DEVICE)
        optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
        criterion = torch.nn.CrossEntropyLoss()

        for epoch in range(5):  # fewer epochs for fast tuning
            train(model, train_loader, optimizer, criterion)

        acc, f1, auc, _ = evaluate(model, test_loader, criterion)
        score = (w_acc * acc) + (w_f1 * f1) + (w_auc * (0 if np.isnan(auc) else auc))
        return score

    start_generation = time.time()
    if use_optuna:
        print("Running Optuna for hyperparameter tuning...")
        study = optuna.create_study(direction="maximize")
        study.optimize(objective, n_trials=n_trials)
        best_params = study.best_params
        print(f"Best hyperparameters: {best_params}")
    else:
      best_params = { "num_layers": 5, "dropout": 0.5, "lr": 0.001, "weight_decay": 1e-4}
      print(f"Using default hyperparameters: {best_params}")

    generation_time = time.time() - start_generation

    # Final Training with best parameters

    print("\nRunning final training GIN...")
    print(best_params)

    model = GIN(num_node_features, hidden_dim, dataset.num_classes,
                num_layers=best_params["num_layers"], dropout=best_params["dropout"]).to(DEVICE)

    optimizer = torch.optim.Adam(model.parameters(), lr=best_params["lr"], weight_decay=best_params["weight_decay"])
    criterion = torch.nn.CrossEntropyLoss()

    history = []
    start_time = time.time()
    eval_acc, eval_f1, eval_auc, eval_loss, eval_epoch = 0, 0, 0, 1e9, 0
    best_loss_for_best_epoch = 1e9
    for epoch in range(1, epochs + 1):
        loss = train(model, train_loader, optimizer, criterion)
        acc, f1, auc, e_loss = evaluate(model, test_loader, criterion)
        if acc > eval_acc:
          #edo mipos to allakso na einai kai edo sindiasmos me weights poy eixe kai sto optuna
          eval_acc, eval_f1, eval_auc, eval_loss, eval_epoch = acc, f1, auc, e_loss, epoch
          best_loss_for_best_epoch = e_loss

        elapsed = time.time() - start_time
        print(f"Epoch {epoch:03d} | Loss={loss:.4f} | TestAcc={acc:.3f} | F1={f1:.3f} | AUC={auc:.3f} | Time={elapsed:.2f}s")
        history.append([epoch, loss, acc, f1, auc, elapsed])

    training_time = time.time() - start_time
    process = psutil.Process(os.getpid())
    memory_usage = process.memory_info().rss / (1024 ** 2)  # in MB


    # Save GIN embeddings for the whole dataset

    full_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
    gin_embeddings, gin_labels = get_gin_embeddings(model, full_loader)

    gin_exp_dir = os.path.join(EMBEDDINGS_DIR, "GIN", dataset_name, experiment_id)
    os.makedirs(gin_exp_dir, exist_ok=True)
    np.save(os.path.join(gin_exp_dir, "embeddings.npy"), gin_embeddings)
    np.save(os.path.join(gin_exp_dir, "labels.npy"), gin_labels)
    # Fix: persist the exact graph order used to produce embeddings/labels
    np.save(os.path.join(gin_exp_dir, "order.npy"), perm)

    # Log file save

    summary_path = f"{CLASSIF_RESULTS_DIR}/gin.csv"
    os.makedirs("results", exist_ok=True)


    summary_data = {
        "method": "GIN",
        "seed": seed,
        "dataset": dataset_name,
        "optimization_enabled": "yes" if use_optuna else "no",
        "embedding_dimension": hidden_dim,
        "objective_weights": f"({w_acc},{w_f1},{w_auc})",
        "num_layers": best_params["num_layers"],
        "dropout": best_params["dropout"],
        "lr": best_params["lr"],
        "weight_decay": best_params["weight_decay"],
        "epochs": epochs,
        "best_epoch": eval_epoch,
        "best_loss": round(float(best_loss_for_best_epoch), 4),
        "eval_loss": round(float(eval_loss), 4),
        "eval_acc": round(eval_acc, 4),
        "eval_f1": round(eval_f1, 4),
        "eval_auc": round(eval_auc, 4),
        "training_time (s)": round(training_time, 2),
        "generation_time (s)": round(generation_time, 2),
        "memory_usage (MB)": round(memory_usage, 2)
    }

    df = pd.DataFrame([summary_data])

    # Append mode (keep all trainings)
    if os.path.exists(summary_path):
        df.to_csv(summary_path, mode='a', index=False, header=False)
    else:
        df.to_csv(summary_path, index=False)

    print(f"\nTraining summary stored in : {summary_path}")
    print(df)

    # Save best model (weights + metadata)
    gin_ckpt = {
        "state_dict": model.state_dict(),
        "num_node_features": num_node_features,
        "hidden_dim": hidden_dim,
        "num_classes": dataset.num_classes,
        "num_layers": best_params["num_layers"],
        "dropout": best_params["dropout"],
    }
    torch.save(gin_ckpt, f"{MODELS_DIR}/GIN_{dataset_name}_{experiment_id}.pth")
    print(f"Saved model: {MODELS_DIR}/GIN_{dataset_name}_{experiment_id}.pth")


In [12]:
def run_graph2vec_pipeline(
    dataset_name,
    seed,
    w_acc=0.5,
    w_f1=0.3,
    w_auc=0.2,
    embedding_dim=128,
    epochs=50,
    test_size=0.2,
    use_optuna=True,
    n_trials=20,
):
    """
    Pipeline for graph classification using Graph2Vec embeddings + SVM,
    with optional Optuna-based hyperparameter tuning and special handling
    for ENZYMES + embedding sanitization.
    """
    # Experiment ID
    experiment_id = str(seed)
    np.random.seed(seed)

    # Load dataset
    # Fix : deterministic dataset order
    dataset_raw = TUDataset(root='data/TUDataset', name=dataset_name)
    perm = make_deterministic_perm(len(dataset_raw), seed)
    dataset = dataset_raw[perm.tolist()]
    print(f"Loaded dataset {dataset_name} for Graph2Vec: {len(dataset)} graphs, {dataset.num_classes} classes")

    # ds_indices[k] is the index in the original (unshuffled) dataset for dataset[k]
    ds_indices = perm.copy()


    # Convert PyG graphs to NetworkX graphs
    graphs = []
    labels = []
    for data in dataset:
        g = to_networkx(data, to_undirected=True)
        graphs.append(g)
        labels.append(int(data.y.item()))

    labels = np.array(labels)
    # Positions in the *current* graphs list (used for splitting)
    pos = np.arange(len(labels))

    # Special handling for ENZYMES (filter very small graphs)
    if dataset_name.upper() == "ENZYMES":
        # Keep original dataset indices aligned with graphs/labels after filtering
        mask = np.array([g.number_of_nodes() >= 3 for g in graphs], dtype=bool)
        graphs = [g for g, keep in zip(graphs, mask) if keep]
        labels = labels[mask]
        ds_indices = ds_indices[mask]
        pos = np.arange(len(labels))
        removed = int(np.sum(~mask))
        print(f"ENZYMES filtering: removed {removed} graphs with < 3 nodes, kept {len(graphs)} graphs.")

    # Outer train/test split on graphs
    train_pos, test_pos, y_train, y_test = train_test_split(
        pos,
        labels,
        test_size=test_size,
        random_state=seed,
        stratify=labels,
    )
    train_graphs = [graphs[i] for i in train_pos]
    test_graphs = [graphs[i] for i in test_pos]

    opt_time = 0.0

    def objective(trial):
        # Hyperparameters for the SVM classifier
        C = trial.suggest_loguniform("C", 1e-2, 1e2)
        gamma = trial.suggest_loguniform("gamma", 1e-4, 1e1)

        # Inner train/validation split on graphs
        inner_tr_idx, inner_val_idx, y_tr, y_val = train_test_split(
            np.arange(len(train_graphs)),
            y_train,
            test_size=0.2,
            random_state=seed,
            stratify=y_train,
        )

        inner_tr_graphs = [train_graphs[i] for i in inner_tr_idx]
        inner_val_graphs = [train_graphs[i] for i in inner_val_idx]


        all_graphs = inner_tr_graphs + inner_val_graphs

        # Fit Graph2Vec on all (transductive setting) and slice embeddings
        g2v = Graph2Vec(dimensions=embedding_dim, wl_iterations=2, epochs=epochs, workers=os.cpu_count())
        g2v.fit(all_graphs)
        emb_all = sanitize_embeddings(g2v.get_embedding())

        X_tr = emb_all[:len(inner_tr_graphs)]
        X_val = emb_all[len(inner_tr_graphs):]

        clf = SVC(kernel="rbf", probability=True, C=C, gamma=gamma, random_state=seed)
        clf.fit(X_tr, y_tr)
        y_pred = clf.predict(X_val)
        y_prob = clf.predict_proba(X_val)

        acc = accuracy_score(y_val, y_pred)
        f1 = f1_score(y_val, y_pred, average="weighted")
        try:
            if len(np.unique(y_val)) == 2:
                auc = roc_auc_score(y_val, y_prob[:, 1])
            else:
                auc = roc_auc_score(y_val, y_prob, multi_class="ovr")
        except ValueError:
            auc = np.nan

        score = (w_acc * acc) + (w_f1 * f1) + (w_auc * (0 if np.isnan(auc) else auc))
        return score

    if use_optuna:
        print("Running Optuna for Graph2Vec+SVM hyperparameter tuning...")
        start_opt = time.time()
        study = optuna.create_study(direction="maximize")
        study.optimize(objective, n_trials=n_trials)
        best_params = study.best_params
        opt_time = time.time() - start_opt
        print(f"Best hyperparameters (Graph2Vec+SVM): {best_params}")
    else:
        best_params = {"C": 1.0, "gamma": "scale"}
        print(f"Using default SVM hyperparameters: {best_params}")

    # Final embedding + training using best hyperparameters
    print("Running final Graph2Vec embedding on train+test graphs...")
    all_graphs_final = train_graphs + test_graphs
    order_pos = np.concatenate([train_pos, test_pos])
    start_embed = time.time()
    g2v = Graph2Vec(dimensions=embedding_dim, wl_iterations=2, epochs=epochs, workers=os.cpu_count())
    g2v.fit(all_graphs_final)
    emb_all = sanitize_embeddings(g2v.get_embedding())
    embed_time = time.time() - start_embed

    X_train = emb_all[:len(train_graphs)]
    X_test = emb_all[len(train_graphs):]

    print("Training final SVM on Graph2Vec embeddings...")
    start_train = time.time()
    clf = SVC(kernel="rbf", probability=True, C=best_params["C"], gamma=best_params["gamma"], random_state=seed)
    clf.fit(X_train, y_train)
    train_time = time.time() - start_train

    # Evaluation on held-out test graphs
    y_pred = clf.predict(X_test)
    y_prob = clf.predict_proba(X_test)

    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average="weighted")
    try:
        if len(np.unique(y_test)) == 2:
            auc = roc_auc_score(y_test, y_prob[:, 1])
        else:
            auc = roc_auc_score(y_test, y_prob, multi_class="ovr")
    except ValueError:
        auc = np.nan

    score = (w_acc * acc) + (w_f1 * f1) + (w_auc * (0 if np.isnan(auc) else auc))


    # Save Graph2Vec embeddings (for all graphs) + labels

    g2v_exp_dir = os.path.join(EMBEDDINGS_DIR, "Graph2Vec", dataset_name, experiment_id)
    os.makedirs(g2v_exp_dir, exist_ok=True)
    np.save(os.path.join(g2v_exp_dir, "embeddings.npy"), emb_all)
    # Save labels in the SAME order as embeddings (train_pos + test_pos)
    np.save(os.path.join(g2v_exp_dir, "labels.npy"), labels[order_pos])
    # Fix: persist ORIGINAL dataset indices (so stability can rebuild graphs correctly)
    np.save(os.path.join(g2v_exp_dir, "order.npy"), ds_indices[order_pos])

    # Save fitted Graph2Vec model
    g2v_model_path = f"{MODELS_DIR}/Graph2Vec_{dataset_name}_{experiment_id}.joblib"
    joblib.dump(g2v, g2v_model_path)

    # Save trained SVM classifier
    svm_model_path = f"{MODELS_DIR}/Graph2Vec_SVM_{dataset_name}_{experiment_id}.joblib"
    joblib.dump(clf, svm_model_path)


    process = psutil.Process(os.getpid())
    memory_usage = process.memory_info().rss / (1024 ** 2)  # in MB

    print(f"Graph2Vec Results on {dataset_name} -> Acc: {acc:.3f}, F1: {f1:.3f}, AUC: {auc:.3f}, Score: {score:.3f}")
    print(f"Embedding time: {embed_time:.2f}s | SVM training time: {train_time:.2f}s | Optuna time: {opt_time:.2f}s | Memory usage: {memory_usage:.2f} MB")

    # Log summary to CSV
    summary_path = f"{CLASSIF_RESULTS_DIR}/g2v.csv"

    summary_data = {
        "method": "Graph2Vec",
        "seed": seed,
        "dataset": dataset_name,
        "embedding_dimension": embedding_dim,
        "optuna_enabled": "yes" if use_optuna else "no",
        "C": best_params["C"],
        "gamma": best_params["gamma"],
        "acc": round(float(acc), 4),
        "f1": round(float(f1), 4),
        "auc": round(float(auc) if not np.isnan(auc) else -1, 4),
        "score": round(float(score), 4),
        "embedding_time (s)": round(embed_time, 2),
        "svm_training_time (s)": round(train_time, 2),
        "optuna_time (s)": round(opt_time, 2),
        "memory_usage (MB)": round(memory_usage, 2),
    }

    df = pd.DataFrame([summary_data])
    if os.path.exists(summary_path):
        df.to_csv(summary_path, mode='a', index=False, header=False)
    else:
        df.to_csv(summary_path, index=False)

    print(f"Graph2Vec summary stored in: {summary_path}")


In [13]:
def run_netlsd_pipeline(
    dataset_name,
    seed,
    w_acc=0.5,
    w_f1=0.3,
    w_auc=0.2,
    test_size=0.2,
    use_optuna=True,
    n_trials=20,
):
    """
    Pipeline for graph classification using NetLSD embeddings + SVM,
    with optional Optuna-based hyperparameter tuning and ENZYMES filtering.
    """
    # Experiment ID
    experiment_id = str(seed)
    np.random.seed(seed)
    # Fix: deterministic dataset order
    dataset_raw = TUDataset(root='data/TUDataset', name=dataset_name)
    perm = make_deterministic_perm(len(dataset_raw), seed)
    dataset = dataset_raw[perm.tolist()]
    print(f"Loaded dataset {dataset_name} for NetLSD: {len(dataset)} graphs, {dataset.num_classes} classes")

    # ds_indices[k] is the index in the original (unshuffled) dataset for dataset[k]
    ds_indices = perm.copy()

    # Convert PyG graphs to NetworkX graphs
    graphs = []
    labels = []
    for data in dataset:
        g = to_networkx(data, to_undirected=True)
        graphs.append(g)
        labels.append(int(data.y.item()))

    labels = np.array(labels)
    pos = np.arange(len(labels))

    # Special handling for ENZYMES
    if dataset_name.upper() == "ENZYMES":
        # Keep original dataset indices aligned with graphs/labels after filtering
        mask = np.array([g.number_of_nodes() >= 3 for g in graphs], dtype=bool)
        graphs = [g for g, keep in zip(graphs, mask) if keep]
        labels = labels[mask]
        ds_indices = ds_indices[mask]
        pos = np.arange(len(labels))
        removed = int(np.sum(~mask))
        print(f"ENZYMES filtering: removed {removed} graphs with < 3 nodes, kept {len(graphs)} graphs.")

    # Outer train/test split on graphs
    train_pos, test_pos, y_train, y_test = train_test_split(
        pos,
        labels,
        test_size=test_size,
        random_state=seed,
        stratify=labels,
    )
    train_graphs = [graphs[i] for i in train_pos]
    test_graphs = [graphs[i] for i in test_pos]

    opt_time = 0.0

    def objective(trial):
        # Hyperparameters for the SVM classifier
        C = trial.suggest_loguniform("C", 1e-2, 1e2)
        gamma = trial.suggest_loguniform("gamma", 1e-4, 1e1)

        # Inner train/validation split on graphs
        inner_tr_idx, inner_val_idx, y_tr, y_val = train_test_split(
            np.arange(len(train_graphs)),
            y_train,
            test_size=0.2,
            random_state=seed,
            stratify=y_train,
        )

        inner_tr_graphs = [train_graphs[i] for i in inner_tr_idx]
        inner_val_graphs = [train_graphs[i] for i in inner_val_idx]


        all_graphs = inner_tr_graphs + inner_val_graphs

        # Fit NetLSD on all and slice embeddings
        netlsd = NetLSD()
        netlsd.fit(all_graphs)
        emb_all = sanitize_embeddings(netlsd.get_embedding())

        X_tr = emb_all[:len(inner_tr_graphs)]
        X_val = emb_all[len(inner_tr_graphs):]

        clf = SVC(kernel="rbf", probability=True, C=C, gamma=gamma, random_state=seed)
        clf.fit(X_tr, y_tr)
        y_pred = clf.predict(X_val)
        y_prob = clf.predict_proba(X_val)

        acc = accuracy_score(y_val, y_pred)
        f1 = f1_score(y_val, y_pred, average="weighted")
        try:
            if len(np.unique(y_val)) == 2:
                auc = roc_auc_score(y_val, y_prob[:, 1])
            else:
                auc = roc_auc_score(y_val, y_prob, multi_class="ovr")
        except ValueError:
            auc = np.nan

        score = (w_acc * acc) + (w_f1 * f1) + (w_auc * (0 if np.isnan(auc) else auc))
        return score

    if use_optuna:
        print("Running Optuna for NetLSD+SVM hyperparameter tuning...")
        start_opt = time.time()
        study = optuna.create_study(direction="maximize")
        study.optimize(objective, n_trials=n_trials)
        best_params = study.best_params
        opt_time = time.time() - start_opt
        print(f"Best hyperparameters (NetLSD+SVM): {best_params}")
    else:
        best_params = {"C": 1.0, "gamma": "scale"}
        print(f"Using default SVM hyperparameters: {best_params}")

    # Final embedding + training using best hyperparameters
    print("Running final NetLSD embedding on train+test graphs...")
    all_graphs_final = train_graphs + test_graphs
    order_pos = np.concatenate([train_pos, test_pos])
    start_embed = time.time()
    netlsd = NetLSD()
    netlsd.fit(all_graphs_final)
    emb_all = sanitize_embeddings(netlsd.get_embedding())
    embed_time = time.time() - start_embed

    X_train = emb_all[:len(train_graphs)]
    X_test = emb_all[len(train_graphs):]

    print("Training final SVM on NetLSD embeddings...")
    start_train = time.time()
    clf = SVC(kernel="rbf", probability=True, C=best_params["C"], gamma=best_params["gamma"], random_state=seed)
    clf.fit(X_train, y_train)
    train_time = time.time() - start_train

    # Evaluation on held-out test graphs
    y_pred = clf.predict(X_test)
    y_prob = clf.predict_proba(X_test)

    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average="weighted")
    try:
        if len(np.unique(y_test)) == 2:
            auc = roc_auc_score(y_test, y_prob[:, 1])
        else:
            auc = roc_auc_score(y_test, y_prob, multi_class="ovr")
    except ValueError:
        auc = np.nan

    score = (w_acc * acc) + (w_f1 * f1) + (w_auc * (0 if np.isnan(auc) else auc))


    # Save NetLSD embeddings (all graphs) + labels

    netlsd_exp_dir = os.path.join(EMBEDDINGS_DIR, "NetLSD",dataset_name, experiment_id)
    os.makedirs(netlsd_exp_dir, exist_ok=True)
    np.save(os.path.join(netlsd_exp_dir, "embeddings.npy"), emb_all)
    # Save labels in the SAME order as embeddings (train_pos + test_pos)
    np.save(os.path.join(netlsd_exp_dir, "labels.npy"), labels[order_pos])
    # Fix: persist ORIGINAL dataset indices (so stability can rebuild graphs correctly)
    np.save(os.path.join(netlsd_exp_dir, "order.npy"), ds_indices[order_pos])

    # Save fitted NetLSD model
    netlsd_model_path = f"{MODELS_DIR}/NetLSD_{dataset_name}_{experiment_id}.joblib"
    joblib.dump(netlsd, netlsd_model_path)

    # Save trained SVM classifier
    svm_model_path = f"{MODELS_DIR}/NetLSD_SVM_{dataset_name}_{experiment_id}.joblib"
    joblib.dump(clf, svm_model_path)


    process = psutil.Process(os.getpid())
    memory_usage = process.memory_info().rss / (1024 ** 2)  # in MB

    print(f"NetLSD Results on {dataset_name} -> Acc: {acc:.3f}, F1: {f1:.3f}, AUC: {auc:.3f}, Score: {score:.3f}")
    print(f"Embedding time: {embed_time:.2f}s | SVM training time: {train_time:.2f}s | Optuna time: {opt_time:.2f}s | Memory usage: {memory_usage:.2f} MB")

    # Log summary to CSV
    summary_path = f"{CLASSIF_RESULTS_DIR}/netlsd.csv"

    summary_data = {
        "method": "NetLSD",
        "seed": seed,
        "optuna_enabled": "yes" if use_optuna else "no",
        "C": best_params["C"],
        "gamma": best_params["gamma"],
        "acc": round(float(acc), 4),
        "f1": round(float(f1), 4),
        "auc": round(float(auc) if not np.isnan(auc) else -1, 4),
        "score": round(float(score), 4),
        "embedding_time (s)": round(embed_time, 2),
        "svm_training_time (s)": round(train_time, 2),
        "optuna_time (s)": round(opt_time, 2),
        "memory_usage (MB)": round(memory_usage, 2),
    }

    df = pd.DataFrame([summary_data])
    if os.path.exists(summary_path):
        df.to_csv(summary_path, mode='a', index=False, header=False)
    else:
        df.to_csv(summary_path, index=False)

    print(f"NetLSD summary stored in: {summary_path}")


In [14]:
for dataset_name in ["IMDB-MULTI", "ENZYMES", "MUTAG"]:
  for seed in [42, 43, 44]:
      run_gin_pipeline(
          dataset_name=dataset_name,
          seed=seed,
          use_optuna=True,
          w_acc=0.5,
          w_f1=0.3,
          w_auc=0.2,
          hidden_dim=64,
          epochs=200,
          batch_size=32,
          n_trials=10,
      )

Dataset has no node features. Applying OneHotDegree transform...


[I 2026-01-27 17:35:32,568] A new study created in memory with name: no-name-8916767e-3b86-4f14-b1ca-cd4d2b0ed20f


Loaded dataset IMDB-MULTI: 1500 graphs, 89 node features, 3 classes
Running Optuna for hyperparameter tuning...


[I 2026-01-27 17:35:39,782] Trial 0 finished with value: 0.44587087521311447 and parameters: {'num_layers': 4, 'dropout': 0.1789115145273103, 'lr': 0.0003149013649881986, 'weight_decay': 8.035259098640091e-05}. Best is trial 0 with value: 0.44587087521311447.
[I 2026-01-27 17:35:44,512] Trial 1 finished with value: 0.4909933557499653 and parameters: {'num_layers': 3, 'dropout': 0.410648645730567, 'lr': 0.0003144417340315641, 'weight_decay': 4.316958953756027e-05}. Best is trial 1 with value: 0.4909933557499653.
[I 2026-01-27 17:35:49,423] Trial 2 finished with value: 0.5029612026679156 and parameters: {'num_layers': 3, 'dropout': 0.594114263810321, 'lr': 0.0021316908204075885, 'weight_decay': 3.663522497433119e-05}. Best is trial 2 with value: 0.5029612026679156.
[I 2026-01-27 17:35:53,164] Trial 3 finished with value: 0.4055543721413451 and parameters: {'num_layers': 4, 'dropout': 0.168177813512488, 'lr': 0.00017266278603552933, 'weight_decay': 0.0001980061603105917}. Best is trial 2 

Best hyperparameters: {'num_layers': 3, 'dropout': 0.594114263810321, 'lr': 0.0021316908204075885, 'weight_decay': 3.663522497433119e-05}

Running final training GIN...
{'num_layers': 3, 'dropout': 0.594114263810321, 'lr': 0.0021316908204075885, 'weight_decay': 3.663522497433119e-05}
Epoch 001 | Loss=2.3996 | TestAcc=0.477 | F1=0.477 | AUC=0.651 | Time=0.35s
Epoch 002 | Loss=1.7556 | TestAcc=0.400 | F1=0.348 | AUC=0.611 | Time=0.70s
Epoch 003 | Loss=1.4624 | TestAcc=0.420 | F1=0.387 | AUC=0.606 | Time=1.05s
Epoch 004 | Loss=1.2173 | TestAcc=0.467 | F1=0.433 | AUC=0.661 | Time=1.40s
Epoch 005 | Loss=1.0912 | TestAcc=0.470 | F1=0.461 | AUC=0.654 | Time=1.77s
Epoch 006 | Loss=1.0943 | TestAcc=0.460 | F1=0.428 | AUC=0.653 | Time=2.12s
Epoch 007 | Loss=1.0525 | TestAcc=0.467 | F1=0.426 | AUC=0.680 | Time=2.46s
Epoch 008 | Loss=1.0355 | TestAcc=0.480 | F1=0.449 | AUC=0.671 | Time=2.82s
Epoch 009 | Loss=1.0368 | TestAcc=0.487 | F1=0.485 | AUC=0.666 | Time=3.17s
Epoch 010 | Loss=1.0314 | TestA

...

Best hyperparameters: {'num_layers': 6, 'dropout': 0.3112402384426334, 'lr': 0.005149715217118113, 'weight_decay': 4.818880269263866e-06}

Running final training GIN...
{'num_layers': 6, 'dropout': 0.3112402384426334, 'lr': 0.005149715217118113, 'weight_decay': 4.818880269263866e-06}
Epoch 001 | Loss=1.1756 | TestAcc=0.711 | F1=0.590 | AUC=0.946 | Time=0.09s
Epoch 002 | Loss=1.4412 | TestAcc=0.711 | F1=0.590 | AUC=0.323 | Time=0.17s
Epoch 003 | Loss=0.8649 | TestAcc=0.237 | F1=0.149 | AUC=0.155 | Time=0.23s
Epoch 004 | Loss=0.6916 | TestAcc=0.289 | F1=0.130 | AUC=0.283 | Time=0.29s
Epoch 005 | Loss=0.4023 | TestAcc=0.132 | F1=0.097 | AUC=0.030 | Time=0.35s
Epoch 006 | Loss=0.3244 | TestAcc=0.263 | F1=0.161 | AUC=0.047 | Time=0.41s
Epoch 007 | Loss=0.3382 | TestAcc=0.263 | F1=0.161 | AUC=0.273 | Time=0.47s
Epoch 008 | Loss=0.4263 | TestAcc=0.316 | F1=0.252 | AUC=0.458 | Time=0.53s
Epoch 009 | Loss=0.4223 | TestAcc=0.842 | F1=0.848 | AUC=0.889 | Time=0.59s
Epoch 010 | Loss=0.4245 | TestA

[I 2026-01-27 17:42:51,989] A new study created in memory with name: no-name-1f1c1a71-ca54-4224-a38f-1d588513eec0


Epoch 200 | Loss=0.2939 | TestAcc=0.842 | F1=0.820 | AUC=0.966 | Time=12.54s

Training summary stored in : /content/drive/MyDrive/InformationSystems/Classification/results/classification/gin.csv
  method  seed dataset optimization_enabled  embedding_dimension  \
0    GIN    43   MUTAG                  yes                   64   

  objective_weights  num_layers  dropout       lr  weight_decay  epochs  \
0     (0.5,0.3,0.2)           6  0.31124  0.00515      0.000005     200   

   best_epoch  best_loss  eval_loss  eval_acc  eval_f1  eval_auc  \
0          23     0.3237     0.3237    0.9211    0.922    0.9596   

   training_time (s)  generation_time (s)  memory_usage (MB)  
0              12.54                 3.01            1440.09  
Saved model: /content/drive/MyDrive/InformationSystems/Classification/models/GIN_MUTAG_43.pth
Loaded dataset MUTAG: 188 graphs, 7 node features, 2 classes
Running Optuna for hyperparameter tuning...


[I 2026-01-27 17:42:52,385] Trial 0 finished with value: 0.7887775998481668 and parameters: {'num_layers': 5, 'dropout': 0.23846767413995934, 'lr': 0.004767743741387555, 'weight_decay': 0.00010172329037934279}. Best is trial 0 with value: 0.7887775998481668.
[I 2026-01-27 17:42:52,731] Trial 1 finished with value: 0.25808779820616007 and parameters: {'num_layers': 6, 'dropout': 0.16272197158301505, 'lr': 0.0005489954961812735, 'weight_decay': 1.7990479666192114e-05}. Best is trial 0 with value: 0.7887775998481668.
[I 2026-01-27 17:42:52,904] Trial 2 finished with value: 0.8552833952697836 and parameters: {'num_layers': 3, 'dropout': 0.4892944312110003, 'lr': 0.002267340128662578, 'weight_decay': 0.00035419932725050293}. Best is trial 2 with value: 0.8552833952697836.
[I 2026-01-27 17:42:53,165] Trial 3 finished with value: 0.7621476706004159 and parameters: {'num_layers': 6, 'dropout': 0.13912176774264445, 'lr': 0.0014417392795328524, 'weight_decay': 0.00016318300583254082}. Best is tr

Best hyperparameters: {'num_layers': 5, 'dropout': 0.5645653497395093, 'lr': 0.005290060438946103, 'weight_decay': 4.172358923310851e-06}

Running final training GIN...
{'num_layers': 5, 'dropout': 0.5645653497395093, 'lr': 0.005290060438946103, 'weight_decay': 4.172358923310851e-06}
Epoch 001 | Loss=1.2848 | TestAcc=0.763 | F1=0.661 | AUC=0.904 | Time=0.05s
Epoch 002 | Loss=0.8336 | TestAcc=0.763 | F1=0.661 | AUC=0.897 | Time=0.10s
Epoch 003 | Loss=1.0150 | TestAcc=0.789 | F1=0.718 | AUC=0.881 | Time=0.15s
Epoch 004 | Loss=0.7314 | TestAcc=0.816 | F1=0.767 | AUC=0.866 | Time=0.20s
Epoch 005 | Loss=0.5198 | TestAcc=0.842 | F1=0.810 | AUC=0.904 | Time=0.25s
Epoch 006 | Loss=0.4884 | TestAcc=0.816 | F1=0.788 | AUC=0.920 | Time=0.31s
Epoch 007 | Loss=0.4197 | TestAcc=0.842 | F1=0.825 | AUC=0.935 | Time=0.36s
Epoch 008 | Loss=0.3670 | TestAcc=0.895 | F1=0.890 | AUC=0.935 | Time=0.41s
Epoch 009 | Loss=0.2968 | TestAcc=0.842 | F1=0.825 | AUC=0.931 | Time=0.47s
Epoch 010 | Loss=0.3331 | TestA

In [15]:
for dataset_name in ["IMDB-MULTI", "ENZYMES", "MUTAG"]:
  for seed in [42, 43, 44]:
      run_graph2vec_pipeline(
          dataset_name=dataset_name,
          seed=seed,
          w_acc=0.5, w_f1=0.3, w_auc=0.2,
          embedding_dim=128,
          epochs=200,
          test_size=0.2,
          use_optuna=True,
          n_trials=10,
      )

Loaded dataset IMDB-MULTI for Graph2Vec: 1500 graphs, 3 classes


[I 2026-01-27 17:43:06,910] A new study created in memory with name: no-name-0658c69d-f2b8-4c8f-940c-48e0a2878913


Running Optuna for Graph2Vec+SVM hyperparameter tuning...


[I 2026-01-27 17:43:16,298] Trial 0 finished with value: 0.47770594407313155 and parameters: {'C': 10.867463624186174, 'gamma': 4.779141986996383}. Best is trial 0 with value: 0.47770594407313155.
[I 2026-01-27 17:43:25,952] Trial 1 finished with value: 0.446353745579119 and parameters: {'C': 65.94454809217339, 'gamma': 3.9232357221947645}. Best is trial 0 with value: 0.47770594407313155.
[I 2026-01-27 17:43:35,237] Trial 2 finished with value: 0.5674344070994178 and parameters: {'C': 0.3155516521751666, 'gamma': 0.0077339853580041485}. Best is trial 2 with value: 0.5674344070994178.
[I 2026-01-27 17:43:44,689] Trial 3 finished with value: 0.4914423534729372 and parameters: {'C': 0.20700711333234584, 'gamma': 2.8270055433898396}. Best is trial 2 with value: 0.5674344070994178.
[I 2026-01-27 17:43:54,009] Trial 4 finished with value: 0.5405306657072796 and parameters: {'C': 3.6792611145200764, 'gamma': 0.000633127285473922}. Best is trial 2 with value: 0.5674344070994178.
[I 2026-01-27 

Best hyperparameters (Graph2Vec+SVM): {'C': 0.3155516521751666, 'gamma': 0.0077339853580041485}
Running final Graph2Vec embedding on train+test graphs...
Training final SVM on Graph2Vec embeddings...
Graph2Vec Results on IMDB-MULTI -> Acc: 0.450, F1: 0.421, AUC: 0.664, Score: 0.484
Embedding time: 11.34s | SVM training time: 1.02s | Optuna time: 95.06s | Memory usage: 1460.57 MB
Graph2Vec summary stored in: /content/drive/MyDrive/InformationSystems/Classification/results/classification/g2v.csv
Loaded dataset IMDB-MULTI for Graph2Vec: 1500 graphs, 3 classes


...

Best hyperparameters (Graph2Vec+SVM): {'C': 27.157199159073894, 'gamma': 0.17646958746853852}
Running final Graph2Vec embedding on train+test graphs...


[I 2026-01-27 17:51:42,079] A new study created in memory with name: no-name-2dbfd664-bf16-40cb-9a41-dfd3d9034780


Training final SVM on Graph2Vec embeddings...
Graph2Vec Results on MUTAG -> Acc: 0.789, F1: 0.789, AUC: 0.880, Score: 0.808
Embedding time: 0.93s | SVM training time: 0.01s | Optuna time: 8.94s | Memory usage: 1470.65 MB
Graph2Vec summary stored in: /content/drive/MyDrive/InformationSystems/Classification/results/classification/g2v.csv
Loaded dataset MUTAG for Graph2Vec: 188 graphs, 2 classes
Running Optuna for Graph2Vec+SVM hyperparameter tuning...


[I 2026-01-27 17:51:42,827] Trial 0 finished with value: 0.793344886606761 and parameters: {'C': 3.415747094029785, 'gamma': 1.556084003590642}. Best is trial 0 with value: 0.793344886606761.
[I 2026-01-27 17:51:43,582] Trial 1 finished with value: 0.7945987232102144 and parameters: {'C': 0.7875087762740153, 'gamma': 3.900645092385029}. Best is trial 1 with value: 0.7945987232102144.
[I 2026-01-27 17:51:44,331] Trial 2 finished with value: 0.6923333333333332 and parameters: {'C': 0.4109401878386158, 'gamma': 0.011465912050758722}. Best is trial 1 with value: 0.7945987232102144.
[I 2026-01-27 17:51:45,090] Trial 3 finished with value: 0.6893333333333334 and parameters: {'C': 1.8958977592663582, 'gamma': 0.0013999152747740937}. Best is trial 1 with value: 0.7945987232102144.
[I 2026-01-27 17:51:45,839] Trial 4 finished with value: 0.6923333333333332 and parameters: {'C': 0.022890984815543306, 'gamma': 0.32112199535638775}. Best is trial 1 with value: 0.7945987232102144.
[I 2026-01-27 17:

Best hyperparameters (Graph2Vec+SVM): {'C': 0.7875087762740153, 'gamma': 3.900645092385029}
Running final Graph2Vec embedding on train+test graphs...
Training final SVM on Graph2Vec embeddings...
Graph2Vec Results on MUTAG -> Acc: 0.789, F1: 0.789, AUC: 0.862, Score: 0.804
Embedding time: 1.35s | SVM training time: 0.01s | Optuna time: 8.28s | Memory usage: 1470.65 MB
Graph2Vec summary stored in: /content/drive/MyDrive/InformationSystems/Classification/results/classification/g2v.csv


In [16]:
for dataset_name in ["IMDB-MULTI", "ENZYMES", "MUTAG"]:
  for seed in [42, 43, 44]:
      run_netlsd_pipeline(
          dataset_name=dataset_name,
          seed=seed,
          w_acc=0.5, w_f1=0.3, w_auc=0.2,
          test_size=0.2,
          use_optuna=True,
          n_trials=10,
      )

Loaded dataset IMDB-MULTI for NetLSD: 1500 graphs, 3 classes


[I 2026-01-27 17:51:52,264] A new study created in memory with name: no-name-d84aa726-f79e-47d5-be99-bea720726354


Running Optuna for NetLSD+SVM hyperparameter tuning...


[I 2026-01-27 17:51:56,644] Trial 0 finished with value: 0.34947103262366186 and parameters: {'C': 0.04033374011187141, 'gamma': 1.54694488280316}. Best is trial 0 with value: 0.34947103262366186.
[I 2026-01-27 17:52:00,935] Trial 1 finished with value: 0.42135489933741077 and parameters: {'C': 0.06192663099172617, 'gamma': 0.00011326495532043095}. Best is trial 1 with value: 0.42135489933741077.
[I 2026-01-27 17:52:06,548] Trial 2 finished with value: 0.4225215660040774 and parameters: {'C': 0.9795645028447804, 'gamma': 0.002972120556553736}. Best is trial 2 with value: 0.4225215660040774.
[I 2026-01-27 17:52:10,804] Trial 3 finished with value: 0.4186047886957813 and parameters: {'C': 0.6116347303349853, 'gamma': 0.040648560150916405}. Best is trial 2 with value: 0.4225215660040774.
[I 2026-01-27 17:52:15,204] Trial 4 finished with value: 0.42261010767074414 and parameters: {'C': 3.460961910612948, 'gamma': 0.0004381137458508494}. Best is trial 4 with value: 0.42261010767074414.
[I 2

Best hyperparameters (NetLSD+SVM): {'C': 9.911160566612544, 'gamma': 2.5794568691187574}
Running final NetLSD embedding on train+test graphs...
Training final SVM on NetLSD embeddings...
NetLSD Results on IMDB-MULTI -> Acc: 0.453, F1: 0.403, AUC: 0.646, Score: 0.477
Embedding time: 5.10s | SVM training time: 1.59s | Optuna time: 46.70s | Memory usage: 1477.66 MB
NetLSD summary stored in: /content/drive/MyDrive/InformationSystems/Classification/results/classification/netlsd.csv
Loaded dataset IMDB-MULTI for NetLSD: 1500 graphs, 3 classes


...

Best hyperparameters (NetLSD+SVM): {'C': 32.85159638528139, 'gamma': 3.1037329792990342}
Running final NetLSD embedding on train+test graphs...


[I 2026-01-27 17:55:54,764] A new study created in memory with name: no-name-cf686a17-11a9-4346-a0e0-36190f239937


Training final SVM on NetLSD embeddings...
NetLSD Results on MUTAG -> Acc: 0.789, F1: 0.778, AUC: 0.831, Score: 0.794
Embedding time: 0.55s | SVM training time: 0.01s | Optuna time: 5.84s | Memory usage: 1503.74 MB
NetLSD summary stored in: /content/drive/MyDrive/InformationSystems/Classification/results/classification/netlsd.csv
Loaded dataset MUTAG for NetLSD: 188 graphs, 2 classes
Running Optuna for NetLSD+SVM hyperparameter tuning...


[I 2026-01-27 17:55:55,211] Trial 0 finished with value: 0.6763333333333332 and parameters: {'C': 2.149651540372503, 'gamma': 0.000773951067036475}. Best is trial 0 with value: 0.6763333333333332.
[I 2026-01-27 17:55:55,672] Trial 1 finished with value: 0.6763333333333332 and parameters: {'C': 0.012819520405551538, 'gamma': 0.04646265411363027}. Best is trial 0 with value: 0.6763333333333332.
[I 2026-01-27 17:55:56,122] Trial 2 finished with value: 0.6763333333333332 and parameters: {'C': 0.0786341955232671, 'gamma': 0.04234625504326544}. Best is trial 0 with value: 0.6763333333333332.
[I 2026-01-27 17:55:56,585] Trial 3 finished with value: 0.6763333333333332 and parameters: {'C': 0.3036874786054862, 'gamma': 0.11250456730688667}. Best is trial 0 with value: 0.6763333333333332.
[I 2026-01-27 17:55:57,034] Trial 4 finished with value: 0.8983994528043775 and parameters: {'C': 20.017512642443723, 'gamma': 0.45360797820022825}. Best is trial 4 with value: 0.8983994528043775.
[I 2026-01-27

Best hyperparameters (NetLSD+SVM): {'C': 9.811077059604782, 'gamma': 1.9642985053025699}
Running final NetLSD embedding on train+test graphs...
Training final SVM on NetLSD embeddings...
NetLSD Results on MUTAG -> Acc: 0.868, F1: 0.870, AUC: 0.862, Score: 0.867
Embedding time: 0.55s | SVM training time: 0.01s | Optuna time: 4.53s | Memory usage: 1503.74 MB
NetLSD summary stored in: /content/drive/MyDrive/InformationSystems/Classification/results/classification/netlsd.csv
